In [1]:
from api import *

In [2]:
url_text = "https://webtris.highwaysengland.co.uk/api/v1/sites"
with urllib.request.urlopen(url_text) as url:
    data = json.loads(url.read().decode())
    
sites = data['sites']

In [3]:
# Specify dates
start_date='10062022'
end_date = '15062022'
# Specify the train area we are looking at
max_lat = 52.50
max_long = -1.67
min_lat = 52.42
min_long = -1.75
birmingham_sites_df = get_quality_area(sites, max_lat, max_long, min_lat, min_long, start_date, end_date)

100%|██████████| 331/331 [01:10<00:00,  4.67it/s]


In [4]:
birmingham_sites_gdf = df_to_gdf(birmingham_sites_df)

In [5]:
birmingham_report_df = get_reports_from_sites_df(birmingham_sites_df, start_date, end_date)

100%|██████████| 55/55 [00:20<00:00,  2.67it/s]


In [10]:
birmingham_report_df = pd.read_feather('traffic_reports/birmingham_report_df')
manc_report_df = pd.read_feather('traffic_reports/manc_report_df')
cam_report_df = pd.read_feather('traffic_reports/cam_report_df')
thorpe_report_df = pd.read_feather('traffic_reports/thorpe_report_df')
epping_report_df = pd.read_feather('traffic_reports/epping_report_df')
bristol_report_df = pd.read_feather('traffic_reports/bristol_report_df')

In [11]:
clean_birmingham_report_df = clean_report(birmingham_report_df)

In [13]:
def normalise(clean_report):
    for name in interger_cols:
        new_name = f"{name}_normalised"
        # for ever row in the report present the row's site id's mean volume
        mean = clean_report.groupby('site_id')[name].transform("mean")
        # normalise
        clean_report.loc[:, new_name] = clean_report[name] / mean
        # filter so we don't have rows with a small mean which causes a pole
    return clean_report[mean>1]


# Specify which cols are to be converted to numerical data
interger_cols = [
     '0-520cm',
     '521-660cm',
     '661-1160cm',
     '1160+cm',
     'total_volume']

clean_birmingham_report_df_norm = normalise(clean_birmingham_report_df)

In [16]:
full_clean_report_norm = pd.read_feather('traffic_reports/clean/full_clean_report_norm')

In [17]:
full_clean_report_norm

,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,...,hour,DOW,month,location,float_time,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
0,M42/6482L,175,2019-03-19T00:00:00,00:14:00,0,19,4,8,18,62,...,0,1,3,birmingham,1.552954e+09,0.154378,0.126290,0.539276,1.035132,0.262072
1,M42/6482L,175,2019-03-19T00:00:00,00:29:00,1,7,4,8,17,57,...,0,1,3,birmingham,1.552955e+09,0.056876,0.126290,0.539276,0.977625,0.192543
2,M42/6482L,175,2019-03-19T00:00:00,00:44:00,2,13,3,5,8,61,...,0,1,3,birmingham,1.552956e+09,0.105627,0.094717,0.337048,0.460059,0.155104
3,M42/6482L,175,2019-03-19T00:00:00,00:59:00,3,17,2,4,20,59,...,0,1,3,birmingham,1.552957e+09,0.138127,0.063145,0.269638,1.150147,0.229982
4,M42/6482L,175,2019-03-19T00:00:00,01:14:00,4,15,6,12,16,61,...,1,1,3,birmingham,1.552958e+09,0.121877,0.189435,0.808914,0.920117,0.262072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26742408,5275/1,7486,2022-04-08T00:00:00,22:59:00,91,125,3,3,16,56,...,22,4,4,bristol,1.649459e+09,0.446043,0.149869,0.186846,0.794521,0.436910
26742409,5275/1,7486,2022-04-08T00:00:00,23:14:00,92,113,6,3,8,57,...,23,4,4,bristol,1.649460e+09,0.403223,0.299738,0.186846,0.397260,0.386383
26742410,5275/1,7486,2022-04-08T00:00:00,23:29:00,93,97,1,5,10,55,...,23,4,4,bristol,1.649461e+09,0.346129,0.049956,0.311410,0.496576,0.335856
26742411,5275/1,7486,2022-04-08T00:00:00,23:44:00,94,84,3,4,9,55,...,23,4,4,bristol,1.649461e+09,0.299741,0.149869,0.249128,0.446918,0.297218
